# Rad7 Package Verification

This notebook demonstrates the usage of the new `rad7` package using example data.

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np

# Add src to path so we can import rad7
sys.path.append('../src')

import rad7
from rad7 import io, processing, plotting, analysis

## 1. Load Data
We load data from the `example_data` directory.

In [ ]:
data_dir = '../example_data'
dfs = io.load_data_from_directory(data_dir)

if dfs:
    raw_df = dfs[0]
    print(f"Loaded DataFrame with {len(raw_df)} rows.")
    display(raw_df.head())
else:
    print("No data found!")

## 2. Process Data
We process the raw data: calculating counts, calibrating, correcting for humidity, and binning.

In [ ]:
bin_size_hours = 1.0

processed_df = processing.process_runs(
    run_list=dfs, 
    bin_hours=bin_size_hours, 
    correct_rh=True
)

display(processed_df.head())

## 3. Visualize
Plot the Radon Concentration over time.

In [ ]:
plotting.plot_concentration(
    processed_df, 
    x_col='DateTime', 
    y_col='RnConc', 
    title=f'Radon Concentration ({bin_size_hours}h Bins)'
)

## 4. Analysis Example
Fit an exponential decay to the data (just as a demonstration).

In [ ]:
# Prepare data for fitting
# Using 'Days' as time axis for numerical stability
start_time = processed_df['DateTime'].iloc[0]
t_data = (processed_df['DateTime'] - start_time).dt.total_seconds().values / 86400.0
y_data = processed_df['RnConc'].values
y_err = processed_df['Uncert_RnConc'].values

# Initial guess: Amp=max, Tau=2 days, Baseline=min
p0 = (np.max(y_data), 2.0, np.min(y_data))

fit_result = analysis.perform_fit(
    t_data, y_data, y_err, p0
)

if fit_result:
    print("Fit Parameters:", fit_result['popt'])
    print("Goodness of Fit:", fit_result['gof'])
    
    plotting.plot_fit(
        t_data, 
        y_data, 
        fit_result['fitted_curve'], 
        y_err, 
        x_label='Days', 
        title='Exponential Fit'
    )